In [101]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, parallel_load_data, get_filename
from func.ml_utils import Classifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_feature_paths = glob('../feature/raw_main/*_train.gz')
feature_names = [re.search(r'/([^/.]*)_train.gz', path).group(1) for path in train_feature_paths]

train_base = read_pkl_gzip('../input/base_train.gz')
Y = train_base[COLUMN_TARGET].values
train_base.drop(COLUMN_TARGET, axis=1, inplace=True)

In [31]:
length = int(len(train_feature)*0.8)
params = {}
metric = 'auc'
model_type = 'lgb'
score_list = []

for feature in tqdm(feature_names):
    if feature in COLUMNS_IGNORE:
        continue
        
    train_feature_path = f'../feature/raw_main/{feature}_train.gz'
    train_feature = read_pkl_gzip(train_feature_path)
    
    x_train = pd.DataFrame(train_feature[:length], columns=[feature])
    y_train = Y[:length]
    x_valid = pd.DataFrame(train_feature[length:], columns=[feature])
    y_valid = Y[length:]

    try:
        score, oof_pred, test_pred, feim, _ = Classifier(
            model_type=model_type,
            x_train=x_train,
            y_train=y_train,
            x_valid=x_valid,
            y_valid=y_valid,
            x_test=[],
            params=params,
        )
    except ValueError:
        continue
    score_list.append([feature, score])


  0%|          | 0/394 [00:00<?, ?it/s]/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 100 rounds.



  0%|          | 1/394 [00:03<21:14,  3.24s/it]

Early stopping, best iteration is:
[1]	valid_0's auc: 0.663637


/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


...

/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 100 rounds.



100%|█████████▉| 393/394 [18:25<00:02,  2.72s/it]

Early stopping, best iteration is:
[98]	valid_0's auc: 0.668967


/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['ProductCD']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/home/yryrgogo/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1225: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['ProductCD']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))

100%|██████████| 394/394 [18:25<00:00,  1.94s/it]


In [33]:
score_result = pd.DataFrame(score_list, columns=['feature', 'score'])
score_result.sort_values(by='score', ascending=False, inplace=True)

In [59]:
np.random.random(y_valid.shape[0])

array([0.23666493, 0.01093762, 0.54521634, ..., 0.30498639, 0.68753627,
       0.32852606])

In [65]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_valid, np.random.random(y_valid.shape[0]))

0.49566674837409075

In [70]:
score_result['score_round'] = score_result['score'].round(2)
score_cnt = score_result['score_round'].value_counts()
score_cnt

0.68    61
0.69    47
0.53    35
0.67    34
0.70    26
0.52    25
0.51    18
0.54    18
0.50    14
0.65    13
0.58    12
0.64    11
0.57    10
0.66     9
0.59     9
0.62     8
0.63     7
0.56     7
0.61     5
0.55     3
0.60     3
0.72     1
0.71     1
Name: score_round, dtype: int64

In [71]:
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

viz = score_result[score_result['score']>0.55]

plt.figure(figsize=(16, 24))
sns.barplot(data=viz, x='score', y='feature')
plt.show()

(265, 3)


SystemExit: 

/home/yryrgogo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [74]:
score_result.to_csv('../eda/103__eda_single_feature_train_AUC_holdout8-2.csv', index=False)

In [87]:
import os
from pathlib import Path
import shutil

def move_feature(list_feature, from_dir, to_dir):
    
    HOME = Path().home()
    DIR_PJ = HOME / 'github/ieee-fraud'
    
    DIR_FROM_FEATURE = Path(f'feature/{from_dir}') 
    DIR_TO_FEATURE   = Path(f'feature/{to_dir}') 
    
    for feature in list_feature:
        from_train_feature_path = str(DIR_PJ / DIR_FROM_FEATURE / f'{feature}_train.gz')
        from_test_feature_path  = str(DIR_PJ / DIR_FROM_FEATURE / f'{feature}_test.gz')
        to_path = str(DIR_PJ / DIR_TO_FEATURE)
        
        shutil.move(from_train_feature_path, to_path)
        shutil.move(from_test_feature_path, to_path)

raw_score_features = score_result[score_result['score']<0.55]['feature'].values
move_feature(raw_score_features, 'raw_main', 'trush')

/home/yryrgogo/github/ieee-fraud/feature/raw_main/V68_train.gz


SystemExit: 

/home/yryrgogo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [102]:
train_paths = glob('../feature/raw_main/*_train.gz')
test_paths = glob('../feature/raw_main/*_test.gz')
feature_names = [get_filename(path) for path in train_paths]

train_df = parallel_load_data(train_paths, feature_names)
print(train_df.shape)

AttributeError: 'list' object has no attribute 'shape'